In [135]:
# Import Dependencies
import pandas as pd
from datetime import datetime as dt
from bs4 import BeautifulSoup as bs
import requests
import os
import numpy as np
from splinter import Browser
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# Google Office Parsing

In [144]:
url='https://about.google/locations/?region=true'
response = requests.get(url)

In [145]:
soup = bs(response.text, 'html.parser')

In [146]:
results = soup.find_all("div", class_= "office-info")
results

[<div class="office-info">
 <h2 class="office-name" itemprop="name">
                   Dubai
                 </h2>
 <div class="office-address" itemprop="address">TECOM Zone, Dubai Internet City
 Dubai, United Arab Emirates</div>
 <div class="office-phone-number">
                     Phone:
                     <span class="phone-number" data-phone-number="+971 4 4509500" itemprop="telephone">
                       +971 4 4509500
                     </span>
 </div>
 <div class="directions">
 <a href="https://www.google.com/maps/dir/Current+Location/25.0929,55.1591?hl=en" rel="noopener" target="_blank">
                     Directions
                   </a>
 </div>
 </div>,
 <div class="office-info">
 <h2 class="office-name" itemprop="name">
                   Haifa
                 </h2>
 <div class="office-address" itemprop="address">Building 30
 MATAM, Advanced Technology Center
 Haifa, 3190500
 Israel </div>
 <div class="office-phone-number">
                     Phone:
      

In [147]:
results = soup.find_all("div", class_= "office-info")
for result in results:
    office = result.find("h2", itemprop= "name").text
    address = result.find("div",itemprop= "address").text
    quards = result.find("a")['href']
    print("-------------------")
    print(office, address, quards)
    
    

-------------------

                  Dubai
                 TECOM Zone, Dubai Internet City
Dubai, United Arab Emirates https://www.google.com/maps/dir/Current+Location/25.0929,55.1591?hl=en
-------------------

                  Haifa
                 Building 30
MATAM, Advanced Technology Center
Haifa, 3190500
Israel  https://www.google.com/maps/dir/Current+Location/32.78897,34.958432?hl=en
-------------------

                  Istanbul
                 Eski Buyukdere Caddesi No: 209
34394
Istanbul, Turkey https://www.google.com/maps/dir/Current+Location/41.081697,29.00859?hl=en
-------------------

                  Johannesburg
                 35 Ballyclare Drive Building E
Johannesburg
2191, South Africa https://www.google.com/maps/dir/Current+Location/-26.0734457,28.032035?hl=en
-------------------

                  Tel Aviv
                 Yigal Alon 98
Tel Aviv, 6789141
Israel  https://www.google.com/maps/dir/Current+Location/32.070043,34.794087?hl=en
-------------------


In [148]:
location = []
addresses = []
quardss = []

for result in results:
    office = result.find("h2", class_= "office-name").text
    address = result.find("div", class_= "office-address").text
    quards = result.find("a")['href']
    location.append(office)
    addresses.append(address)
    quardss.append(quards)
    

In [149]:
# Create a dataframe from the parsed data above

df = pd.DataFrame({
    "Office": location,
    "Address": addresses,
    "Quards": quardss
})

# Removes all occurances of "\n"
df["Office"].replace(r'\s+|\\n', ' ', regex=True, inplace=True) 
df["Address"].replace(r'\s+|\\n', ' ', regex=True, inplace=True) 

#Drops the:'https://www.google.com/maps/dir/Current+Location/ at the beggining of the string
# and the: ?hl=en at the end of the string
df["Quards2"]= df["Quards"].str.strip('https://www.google.com/maps/dir/Current+Location/ ?hl=en')

# Drops the old Column with the google hyperlinks
df = df.drop(columns=['Quards'])


# new data frame with split value columns 
new = df["Quards2"].str.split(",", n = 1, expand = True) 
  
# making separate Latitude column from new data frame 
df["Latitude"]= new[0] 
  
# making separate Longitude column from new data frame 
df["Longitude"]= new[1] 
  
# Dropping old Quards2 columns 
df.drop(columns =["Quards2"], inplace = True) 
  
# Drops duplicates and only keeps the Office's first occurance
df= df.drop_duplicates(subset=["Office"], keep='first')

In [150]:
df["Company"] = ("Google")

In [151]:
df.set_index("Company")

,Office,Address,Latitude,Longitude
Company,,,,
Google,Dubai,"TECOM Zone, Dubai Internet City Dubai, United ...",25.0929,55.1591
Google,Haifa,"Building 30 MATAM, Advanced Technology Center ...",32.78897,34.958432
Google,Istanbul,"Eski Buyukdere Caddesi No: 209 34394 Istanbul,...",41.081697,29.00859
Google,Johannesburg,35 Ballyclare Drive Building E Johannesburg 21...,-26.0734457,28.032035
Google,Tel Aviv,"Yigal Alon 98 Tel Aviv, 6789141 Israel",32.070043,34.794087
...,...,...,...,...
Google,San Francisco,"345 Spear Street San Francisco, CA 94105 Unite...",37.789972,-122.390013
Google,Seattle,"601 N. 34th Street Seattle, WA 98103 United St...",47.649316,-122.350629
Google,Sunnyvale,"803 11th Avenue Sunnyvale, CA 94089 United States",37.403694,-122.031583


# Amazon Office Parsing

In [7]:
executable_path = {'executable_path': "chromedriver.exe"}
browser = Browser('chrome', **executable_path, headless=False)

In [8]:
amazonurl='https://craft.co/amazon/locations?page='
pg_num = ['2', '3', '4', '5', '6', '7', '8']

In [9]:
html = browser.html

In [10]:
amazonsoup = bs(html, "html.parser")
print(amazonsoup.prettify())

<html>
 <head>
 </head>
 <body>
 </body>
</html>


In [11]:
for num in pg_num:
    amazonresults = amazonsoup.find_all('div', itemprop="location")
    for amazonresult in amazonresults:
        office = amazonresult.find(itemprop= "addressLocality").text
        address = amazonresult.find(itemprop="streetAddress").text
        state = amazonresult.find(class_="location__state").text
        country = amazonresult.find(class_="location__country").text
        print("-------------------")
        print(num, office, address, state, country)   

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

master_df=[]


url='https://craft.co/amazon/locations?page='
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-notifications")
driver=webdriver.Chrome('chromedriver.exe',
                 chrome_options=webdriver.ChromeOptions().add_argument("--disable-notifications"))
for i in range (1, 9):
    url_visit=url+str(i)
    driver.get(url_visit)
    try:
        driver.find_element_by_class_name('craft-modal_close_button').click()
    except:
        pass
    
    try:
        driver.find_element_by_class_name('auth-form__link').click()
        username = driver.find_element_by_name("user[email]")
        password = driver.find_element_by_name("user[password]")
        username.send_keys("******")
        password.send_keys("*******")
        driver.find_element_by_class_name('auth-form__actions').click()
    except:
        pass
    soup=BeautifulSoup(driver.page_source, 'lxml')
    locations=soup.find_all("div", class_= "location")
    
    for i, amazonresult in enumerate(locations):
        location_dict={}
        try:
            #print(amazonresult)
            location_dict['Office'] = amazonresult.find(itemprop= "addressLocality").text
            location_dict['Address'] = amazonresult.find(itemprop="streetAddress")
            location_dict['State'] = amazonresult.find(class_="location__state")
            location_dict['Country'] = amazonresult.find(class_="location__country").text
#             print(office,address,state,country)
            master_df.append(location_dict)
        except:
            pass

    
    
master_df

In [129]:
Amazon_locations= pd.DataFrame(master_df)

In [130]:
Amazon_locations

,Office,Address,State,Country
0,Seattle,[410 Terry Ave N],[WA],United States
1,Arlington,[Crystal City],[VA],United States
2,Atlanta,[3333 Piedmont Rd NE],[GA],United States
3,Austin,[11501 Alterra Pkwy],[TX],United States
4,Baltimore,[2010 Broening Hwy],[MD],United States
...,...,...,...,...
228,Rugeley,"[tower business park, Power Station Rd]",None,United Kingdom
229,Severn Beach,[Western Approach Distribution Park],None,United Kingdom
230,Skewen,[Ffordd Amazon],None,United Kingdom
231,Sutton-in-Ashfield,[Sherwood Way S],None,United Kingdom


In [152]:
Amazon_locations["Company"] = ("Amazon")

In [153]:
Amazon_locations.set_index("Company")

,Office,Address,State,Country
Company,,,,
Amazon,Seattle,[410 Terry Ave N],[WA],United States
Amazon,Arlington,[Crystal City],[VA],United States
Amazon,Atlanta,[3333 Piedmont Rd NE],[GA],United States
Amazon,Austin,[11501 Alterra Pkwy],[TX],United States
Amazon,Baltimore,[2010 Broening Hwy],[MD],United States
...,...,...,...,...
Amazon,Rugeley,"[tower business park, Power Station Rd]",None,United Kingdom
Amazon,Severn Beach,[Western Approach Distribution Park],None,United Kingdom
Amazon,Skewen,[Ffordd Amazon],None,United Kingdom


# Apple Parsing

In [178]:
executable_path = {'executable_path': "chromedriver.exe"}
browser = Browser('chrome', **executable_path, headless=False)

appleurl= 'https://craft.co/apple/locations'
browser.visit(appleurl)

html = browser.html

applesoup = bs(html, "html.parser")

appleoffice = []

appleresults = applesoup.find_all("div", class_= "location")

for appleresult in appleresults:
    location_dict={}
    try:
        location_dict['Office'] = appleresult.find(itemprop= "addressLocality").text
        location_dict['Address'] = appleresult.find(itemprop="streetAddress")
        location_dict['State'] = appleresult.find(class_="location__state")
        location_dict['Country'] = appleresult.find(class_="location__country").text
        appleoffice.append(location_dict)
    except:
        pass

In [179]:
appleoffice

[{'Office': 'Cupertino',
  'Address': <span itemprop="streetAddress">One Apple Park Way</span>,
  'State': <span class="location__state">CA</span>,
  'Country': 'United States'},
 {'Office': 'Austin',
  'Address': <span itemprop="streetAddress">12801 Delcour Dr</span>,
  'State': <span class="location__state">TX</span>,
  'Country': 'United States'},
 {'Office': 'Cupertino',
  'Address': <span itemprop="streetAddress">1 Infinite Loop</span>,
  'State': <span class="location__state">CA</span>,
  'Country': 'United States'},
 {'Office': 'Sydney',
  'Address': <span itemprop="streetAddress">Level 3/20 Martin Pl</span>,
  'State': None,
  'Country': 'Australia'},
 {'Office': 'Toronto',
  'Address': <span itemprop="streetAddress">1600 120 Bremner Blvd</span>,
  'State': None,
  'Country': 'Canada'},
 {'Office': 'Las Condes',
  'Address': <span itemprop="streetAddress">Cerro El Plomo 5630</span>,
  'State': None,
  'Country': 'Chile'},
 {'Office': 'Shanghai',
  'Address': <span itemprop="str

In [180]:
apple_locations = pd.DataFrame(appleoffice)

In [181]:
apple_locations

,Office,Address,State,Country
0,Cupertino,[One Apple Park Way],[CA],United States
1,Austin,[12801 Delcour Dr],[TX],United States
2,Cupertino,[1 Infinite Loop],[CA],United States
3,Sydney,[Level 3/20 Martin Pl],None,Australia
4,Toronto,[1600 120 Bremner Blvd],None,Canada
5,Las Condes,[Cerro El Plomo 5630],None,Chile
6,Shanghai,"[Building 6, Block C, 88 Maji Road]",None,China
7,Paris,[7 Place d'Iéna],None,France
8,Hong Kong,"[1 Matheson St, Causeway Bay]",None,Hong Kong
9,Bengaluru,[UB City No.24 Vittal Mallya Rd],None,India


In [182]:
apple_locations["Company"] = ("Apple")
apple_locations.set_index("Company")

,Office,Address,State,Country
Company,,,,
Apple,Cupertino,[One Apple Park Way],[CA],United States
Apple,Austin,[12801 Delcour Dr],[TX],United States
Apple,Cupertino,[1 Infinite Loop],[CA],United States
Apple,Sydney,[Level 3/20 Martin Pl],None,Australia
Apple,Toronto,[1600 120 Bremner Blvd],None,Canada
Apple,Las Condes,[Cerro El Plomo 5630],None,Chile
Apple,Shanghai,"[Building 6, Block C, 88 Maji Road]",None,China
Apple,Paris,[7 Place d'Iéna],None,France
Apple,Hong Kong,"[1 Matheson St, Causeway Bay]",None,Hong Kong


# Micrsoft Parsing

In [141]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

master_df=[]


url='https://craft.co/microsoft/locations?page='
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-notifications")
driver=webdriver.Chrome('chromedriver.exe',
                 chrome_options=webdriver.ChromeOptions().add_argument("--disable-notifications"))
for i in range (1, 7):
    url_visit=url+str(i)
    driver.get(url_visit)
    try:
        driver.find_element_by_class_name('craft-modal_close_button').click()
    except:
        pass
    
    try:
        driver.find_element_by_class_name('auth-form__link').click()
        username = driver.find_element_by_name("user[email]")
        password = driver.find_element_by_name("user[password]")
        username.send_keys("******")
        password.send_keys("*******"))
        driver.find_element_by_class_name('auth-form__actions').click()
    except:
        pass
    soup=BeautifulSoup(driver.page_source, 'lxml')
    locations=soup.find_all("div", class_= "location")
    
    for i, microresult in enumerate(locations):
        location_dict={}
        try:
            #print(amazonresult)
            location_dict['Office'] = microresult.find(itemprop= "addressLocality").text
            location_dict['Address'] = microresult.find(itemprop="streetAddress")
            location_dict['State'] = microresult.find(class_="location__state")
            location_dict['Country'] = microresult.find(class_="location__country").text
#             print(office,address,state,country)
            master_df.append(location_dict)
        except:
            pass

    
    
master_df

[{'Office': 'Redmond',
  'Address': <span itemprop="streetAddress">Microsoft Corporation
  One Microsoft Way</span>,
  'State': <span class="location__state">WA</span>,
  'Country': 'United States'},
 {'Office': 'Albany',
  'Address': <span itemprop="streetAddress">54 State St 7th Floor Suite 701</span>,
  'State': <span class="location__state">NY</span>,
  'Country': 'United States'},
 {'Office': 'Alpharetta',
  'Address': <span itemprop="streetAddress">8000 Avalon Blvd #800</span>,
  'State': <span class="location__state">GA</span>,
  'Country': 'United States'},
 {'Office': 'Atlanta',
  'Address': <span itemprop="streetAddress">84 Peachtree St</span>,
  'State': <span class="location__state">GA</span>,
  'Country': 'United States'},
 {'Office': 'Austin',
  'Address': <span itemprop="streetAddress">10900 Stonelake Blvd #225</span>,
  'State': <span class="location__state">TX</span>,
  'Country': 'United States'},
 {'Office': 'Bellevue',
  'Address': <span itemprop="streetAddress">700

In [170]:
micrsoft_location = pd.DataFrame(master_df)

In [171]:
micrsoft_location

,Office,Address,State,Country
0,Redmond,[Microsoft Corporation\r\nOne Microsoft Way],[WA],United States
1,Albany,[54 State St 7th Floor Suite 701],[NY],United States
2,Alpharetta,[8000 Avalon Blvd #800],[GA],United States
3,Atlanta,[84 Peachtree St],[GA],United States
4,Austin,[10900 Stonelake Blvd #225],[TX],United States
...,...,...,...,...
172,Belfast,"[The Innovation Centre, Queens Road, Queens Is...",None,United Kingdom
173,Wokingham,[Thames Valley Park Reading Berkshire ],None,United Kingdom
174,Montevideo,[Mercosur Business Center Cebollati 1474 - Pis...,None,Uruguay
175,Caracas,[Av. Principal de La Castellana Centro Letonia...,None,Venezuela (Bolivarian Republic of)


In [173]:
micrsoft_location["Company"] = ("Microsoft")
micrsoft_location.set_index("Company")

,Office,Address,State,Country
Company,,,,
Microsoft,Redmond,[Microsoft Corporation\r\nOne Microsoft Way],[WA],United States
Microsoft,Albany,[54 State St 7th Floor Suite 701],[NY],United States
Microsoft,Alpharetta,[8000 Avalon Blvd #800],[GA],United States
Microsoft,Atlanta,[84 Peachtree St],[GA],United States
Microsoft,Austin,[10900 Stonelake Blvd #225],[TX],United States
...,...,...,...,...
Microsoft,Belfast,"[The Innovation Centre, Queens Road, Queens Is...",None,United Kingdom
Microsoft,Wokingham,[Thames Valley Park Reading Berkshire ],None,United Kingdom
Microsoft,Montevideo,[Mercosur Business Center Cebollati 1474 - Pis...,None,Uruguay


# Netflix Parsing

In [188]:
executable_path = {'executable_path': "chromedriver.exe"}
browser = Browser('chrome', **executable_path, headless=False)

netflixurl= 'https://craft.co/netflix/locations'
browser.visit(netflixurl)

html = browser.html

netflixsoup = bs(html, "html.parser")

netflixoffice = []

netlfixresults = netflixsoup.find_all("div", class_= "location")

for netlfixresult in netlfixresults:
    location_dict={}
    try:
        location_dict['Office'] = netlfixresult.find(itemprop= "addressLocality").text
        location_dict['Address'] = netlfixresult.find(itemprop="streetAddress")
        location_dict['State'] = netlfixresult.find(class_="location__state")
        location_dict['Country'] = netlfixresult.find(class_="location__country").text
        netflixoffice.append(location_dict)
    except:
        pass
netflixoffice

NameError: name 'netflix_office' is not defined

In [189]:
netflix_location = pd.DataFrame(netflixoffice)

In [190]:
netflix_location

,Office,Address,State,Country
0,Los Gatos,[100 Winchester Cir],[CA],United States
1,Los Angeles,[5808 Sunset Blvd],[CA],United States
2,New York,[],[NY],United States
3,Washington,[],[DC],United States
4,Paris,[148 Rue de l'Université],None,France
5,Berlin,"[DE-BER-FRS, Friedrichstraße 88]",None,Germany
6,Mexico City,[],None,Mexico
7,Amsterdam,[Karperstraat 8],None,Netherlands
8,Amsterdam,[Stadhouderskade 55],None,Netherlands
9,Manila,[],None,Philippines


In [191]:
netflix_location["Company"] = ("Netflix")
netflix_location.set_index("Company")

,Office,Address,State,Country
Company,,,,
Netflix,Los Gatos,[100 Winchester Cir],[CA],United States
Netflix,Los Angeles,[5808 Sunset Blvd],[CA],United States
Netflix,New York,[],[NY],United States
Netflix,Washington,[],[DC],United States
Netflix,Paris,[148 Rue de l'Université],None,France
Netflix,Berlin,"[DE-BER-FRS, Friedrichstraße 88]",None,Germany
Netflix,Mexico City,[],None,Mexico
Netflix,Amsterdam,[Karperstraat 8],None,Netherlands
Netflix,Amsterdam,[Stadhouderskade 55],None,Netherlands


# Facebook Parsing

In [214]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

master_df=[]


url='https://craft.co/facebook/locations?page='
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-notifications")
driver=webdriver.Chrome('chromedriver.exe',
                 chrome_options=webdriver.ChromeOptions().add_argument("--disable-notifications"))
for i in range (1, 4):
    url_visit=url+str(i)
    driver.get(url_visit)
    try:
        driver.find_element_by_class_name('craft-modal_close_button').click()
    except:
        pass
    
    try:
        driver.find_element_by_class_name('auth-form__link').click()
        username = driver.find_element_by_name("user[email]")
        password = driver.find_element_by_name("user[password]")
        username.send_keys("******")
        password.send_keys("*******")
        driver.find_element_by_class_name('auth-form__actions').click()
    except:
        pass
    soup=BeautifulSoup(driver.page_source, 'lxml')
    locations=soup.find_all("div", class_= "location")
    
    for i, fbresult in enumerate(locations):
        location_dict={}
        try:
            #print(amazonresult)
            location_dict['Office'] = fbresult.find(itemprop= "addressLocality").text
            location_dict['Address'] = fbresult.find(itemprop="streetAddress")
            location_dict['State'] = fbresult.find(class_="location__state")
            location_dict['Country'] = fbresult.find(class_="location__country").text
#             print(office,address,state,country)
            master_df.append(location_dict)
        except:
            pass

    
    
master_df

[{'Office': 'Menlo Park',
  'Address': <span itemprop="streetAddress">1 Hacker Way #15</span>,
  'State': <span class="location__state">CA</span>,
  'Country': 'United States'},
 {'Office': 'Altoona',
  'Address': <span itemprop="streetAddress">100 Share Way NW</span>,
  'State': <span class="location__state">IA</span>,
  'Country': 'United States'},
 {'Office': 'Austin',
  'Address': <span itemprop="streetAddress">300 W 6th St</span>,
  'State': <span class="location__state">TX</span>,
  'Country': 'United States'},
 {'Office': 'Cambridge',
  'Address': <span itemprop="streetAddress">100 Binney St</span>,
  'State': <span class="location__state">MA</span>,
  'Country': 'United States'},
 {'Office': 'Chicago',
  'Address': <span itemprop="streetAddress">191 North Wacker, 191 N Upper Wacker Dr</span>,
  'State': <span class="location__state">IL</span>,
  'Country': 'United States'},
 {'Office': 'Denver',
  'Address': <span itemprop="streetAddress">1900 16th St Mall</span>,
  'State': <s

In [215]:
fb_location = pd.DataFrame(master_df)

In [217]:
fb_location

,Office,Address,State,Country
0,Menlo Park,[1 Hacker Way #15],[CA],United States
1,Altoona,[100 Share Way NW],[IA],United States
2,Austin,[300 W 6th St],[TX],United States
3,Cambridge,[100 Binney St],[MA],United States
4,Chicago,"[191 North Wacker, 191 N Upper Wacker Dr]",[IL],United States
...,...,...,...,...
80,Zürich,[Giesshübelstrasse 30],None,Switzerland
81,Taipei,[],None,"Taiwan, Province of China"
82,Bangkok,"[Gaysorn Tower, Lumphini, Pathum Wan District]",None,Thailand
83,Dubai,[],None,United Arab Emirates


In [218]:
fb_location["Company"] = ("Facebook")
fb_location.set_index("Company")

,Office,Address,State,Country
Company,,,,
Facebook,Menlo Park,[1 Hacker Way #15],[CA],United States
Facebook,Altoona,[100 Share Way NW],[IA],United States
Facebook,Austin,[300 W 6th St],[TX],United States
Facebook,Cambridge,[100 Binney St],[MA],United States
Facebook,Chicago,"[191 North Wacker, 191 N Upper Wacker Dr]",[IL],United States
...,...,...,...,...
Facebook,Zürich,[Giesshübelstrasse 30],None,Switzerland
Facebook,Taipei,[],None,"Taiwan, Province of China"
Facebook,Bangkok,"[Gaysorn Tower, Lumphini, Pathum Wan District]",None,Thailand


# Google Parsing

In [197]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

master_df=[]


url='https://craft.co/google/locations?page='
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-notifications")
driver=webdriver.Chrome('chromedriver.exe',
                 chrome_options=webdriver.ChromeOptions().add_argument("--disable-notifications"))
for i in range (1, 4):
    url_visit=url+str(i)
    driver.get(url_visit)
    try:
        driver.find_element_by_class_name('craft-modal_close_button').click()
    except:
        pass
    
    try:
        driver.find_element_by_class_name('auth-form__link').click()
        username = driver.find_element_by_name("user[email]")
        password = driver.find_element_by_name("user[password]")
        username.send_keys("******")
        password.send_keys("*******")
        driver.find_element_by_class_name('auth-form__actions').click()
    except:
        pass
    soup=BeautifulSoup(driver.page_source, 'lxml')
    locations=soup.find_all("div", class_= "location")
    
    for i, googresult in enumerate(locations):
        location_dict={}
        try:
            #print(amazonresult)
            location_dict['Office'] = googresult.find(itemprop= "addressLocality").text
            location_dict['Address'] = googresult.find(itemprop="streetAddress")
            location_dict['State'] = googresult.find(class_="location__state")
            location_dict['Country'] = googresult.find(class_="location__country").text
#             print(office,address,state,country)
            master_df.append(location_dict)
        except:
            pass

    
    
master_df

[{'Office': 'Mountain View',
  'Address': <span itemprop="streetAddress">1600 Amphitheatre Pkwy</span>,
  'State': <span class="location__state">CA</span>,
  'Country': 'United States'},
 {'Office': 'Ann Arbor',
  'Address': <span itemprop="streetAddress">2300 Traverwood Dr</span>,
  'State': <span class="location__state">MI</span>,
  'Country': 'United States'},
 {'Office': 'Atlanta',
  'Address': <span itemprop="streetAddress">10 10th St NE</span>,
  'State': <span class="location__state">GA</span>,
  'Country': 'United States'},
 {'Office': 'Austin',
  'Address': <span itemprop="streetAddress">500 W 2nd St #2900</span>,
  'State': <span class="location__state">TX</span>,
  'Country': 'United States'},
 {'Office': 'Boulder',
  'Address': <span itemprop="streetAddress">2600 Pearl St</span>,
  'State': <span class="location__state">CO</span>,
  'Country': 'United States'},
 {'Office': 'Boulder',
  'Address': <span itemprop="streetAddress">2930 Pearl St</span>,
  'State': <span class="l

In [198]:
google_location = pd.DataFrame(master_df)

In [199]:
google_location

,Office,Address,State,Country
0,Mountain View,[1600 Amphitheatre Pkwy],[CA],United States
1,Ann Arbor,[2300 Traverwood Dr],[MI],United States
2,Atlanta,[10 10th St NE],[GA],United States
3,Austin,[500 W 2nd St #2900],[TX],United States
4,Boulder,[2600 Pearl St],[CO],United States
...,...,...,...,...
80,Istanbul,[Eski Buyukdere Caddesi No: 209],None,Turkey
81,Dubai,"[TECOM Zone, Dubai Internet City]",None,United Arab Emirates
82,London,[1-13 St Giles High St],None,United Kingdom
83,London,[6 Pancras Square],None,United Kingdom


In [200]:
google_location["Company"] = ("Google")
google_location.set_index("Company")

,Office,Address,State,Country
Company,,,,
Google,Mountain View,[1600 Amphitheatre Pkwy],[CA],United States
Google,Ann Arbor,[2300 Traverwood Dr],[MI],United States
Google,Atlanta,[10 10th St NE],[GA],United States
Google,Austin,[500 W 2nd St #2900],[TX],United States
Google,Boulder,[2600 Pearl St],[CO],United States
...,...,...,...,...
Google,Istanbul,[Eski Buyukdere Caddesi No: 209],None,Turkey
Google,Dubai,"[TECOM Zone, Dubai Internet City]",None,United Arab Emirates
Google,London,[1-13 St Giles High St],None,United Kingdom


In [206]:
Amazon_locations.to_csv(index=False)
compression_opts = dict(method='zip',
                        archive_name='amazon.csv')  
Amazon_locations.to_csv('amazon.zip', index=False,
          compression=compression_opts)

In [203]:
google_location.to_csv(index=False)
compression_opts = dict(method='zip',
                        archive_name='google.csv')  
google_location.to_csv('google.zip', index=False,
          compression=compression_opts) 

In [207]:
netflix_location.to_csv(index=False)
compression_opts = dict(method='zip',
                        archive_name='netflix.csv')  
netflix_location.to_csv('netflix.zip', index=False,
          compression=compression_opts) 

In [211]:
micrsoft_location.to_csv(index=False)
compression_opts = dict(method='zip',
                        archive_name='microsoft.csv')  
micrsoft_location.to_csv('microsoft.zip', index=False,
          compression=compression_opts) 

In [213]:
apple_locations.to_csv(index=False)
compression_opts = dict(method='zip',
                        archive_name='apple.csv')  
apple_locations.to_csv('apple.zip', index=False,
          compression=compression_opts) 

In [219]:
fb_location.to_csv(index=False)
compression_opts = dict(method='zip',
                        archive_name='facebook.csv')  
fb_location.to_csv('facebook.zip', index=False,
          compression=compression_opts) 

In [220]:
df.to_csv(index=False)
compression_opts = dict(method='zip',
                        archive_name='googdf.csv')  
df.to_csv('df.zip', index=False,
          compression=compression_opts) 